# Training

In [1]:
from torch.utils.data import DataLoader
import gc
import torch
from tqdm.notebook import tqdm
import math

from detr.config import config
from augmentations.aug import get_augmentor
from detr.utils.data import WheatDataset, collate_fn
#from fasterrcnn.utils.training_handler import get_training_handler

In [2]:
conf = config()
augmentor = get_augmentor(min_visibility=0.3, blur = [0., 5], gaussian_noise = [0.15, 5/255], rgb_shift = [0.2, 15/255], dropout = 0.1)

#conf.DATA_PATH = os.path.join('/kaggle','input', 'global-wheat-detection')
conf.BATCH_SIZE = 4
conf.DEVICE = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
#conf.SPLIT = 0.8

WD_Train = WheatDataset(conf, 
                  is_train=True, 
                  augmentation=True,
                  normalize=True,
                  augmentor=augmentor, 
                  random_seed=0)

WD_Train_Loader = DataLoader(WD_Train, batch_size=conf.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)


In [6]:
import torch.nn.functional as F
from torch import nn

EPOCH = 0

GWD_detr_Model = torch.hub.load('facebookresearch/detr', 'detr_resnet101', pretrained=False, num_classes=1)


checkpoint = torch.hub.load_state_dict_from_url(
            url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
            #map_location='cpu',
            check_hash=True)
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]
GWD_detr_Model.load_state_dict(checkpoint["model"], strict=False)


_ = GWD_detr_Model.to(conf.DEVICE)

Using cache found in C:\Users\jay/.cache\torch\hub\facebookresearch_detr_master


In [8]:
#print(GWD_detr_Model)
params = [p for p in GWD_detr_Model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.00001, weight_decay=0.0001)  

back = []
front=[]
for k, v in GWD_detr_Model.named_parameters():
    if 'backbone.' in k:
        back.append(v)
    else:
        front.append(v)
lr = 1e-4
optimizer = torch.optim.SGD([{'params': back, 'lr': lr/10}, {'params': front}], lr=lr, weight_decay=0.0001)         

In [14]:
from detr.detr.models.detr import SetCriterion
from detr.detr.models.matcher import HungarianMatcher

num_classes=1

#These are the default values in the source code
bbox_loss_coef=5.0 
giou_loss_coef=2.0
eos_coef=0.1
set_cost_class=1.0
set_cost_bbox=5.0
set_cost_giou=2.0


matcher = HungarianMatcher(cost_class=set_cost_class, cost_bbox=set_cost_bbox, cost_giou=set_cost_giou)
weight_dict = {'loss_ce': 1, 'loss_bbox': bbox_loss_coef}
weight_dict['loss_giou'] = giou_loss_coef
losses = ['labels', 'boxes', 'cardinality']

criterion = SetCriterion(num_classes, matcher=matcher, weight_dict=weight_dict,eos_coef=eos_coef, losses=losses)
_ = criterion.to(conf.DEVICE)

In [15]:
_iter=len(WD_Train_Loader)*(EPOCH)
clip_max_norm=0.1  #Default value in their code

_ = GWD_detr_Model.train()
_ = criterion.train()

for i in tqdm(range(EPOCH, 50, 1)):
    ep_loss_ce=0
    ep_class_error=0
    ep_loss_bbox=0 
    ep_loss_giou=0 
    ep_cardinality_error=0
    
    
    for images, targets in tqdm(WD_Train_Loader):

        images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
        targets = [{k: torch.tensor(v).to(conf.DEVICE) for k, v in target.items()} for target in targets]
        #print(targets[0]['labels'])
        
        #Main fwd pass and loss calc  
        outputs = GWD_detr_Model(images)
        
        #print(outputs['pred_logits'][0])

        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict
        summed_loss_value = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        if math.isfinite(summed_loss_value):
            optimizer.zero_grad()
            summed_loss_value.backward()
            if clip_max_norm > 0:
                torch.nn.utils.clip_grad_norm_(GWD_detr_Model.parameters(), clip_max_norm)
            optimizer.step()
            
            ep_loss_ce+= loss_dict['loss_ce'].item()
            ep_class_error+= loss_dict['class_error'].item()
            ep_loss_bbox+= loss_dict['loss_bbox'].item()
            ep_loss_giou+= loss_dict['loss_giou'].item()
            ep_cardinality_error+= loss_dict['cardinality_error'].item()
            
            if(_iter%50 == 0):
                print("".join([k+":"+str(v.data.cpu().numpy())+", " for k,v in loss_dict.items()]))
    

        else:
            print('Loss is undefined:',summed_loss_value,'   skipping BackProp for step no:',_iter)
            print(loss_dict)
            
        _iter+=1
       
    #Divide by number of batches
    ep_loss_ce = float(ep_loss_ce/len(WD_Train_Loader))  
    ep_class_error = float(ep_class_error/len(WD_Train_Loader))
    ep_loss_bbox = float(ep_loss_bbox/len(WD_Train_Loader))
    ep_loss_giou = float(ep_loss_giou/len(WD_Train_Loader))
    ep_cardinality_error = float(ep_cardinality_error/len(WD_Train_Loader))
    _epoch_loss = ep_loss_ce+ep_class_error+ep_loss_bbox+ep_loss_giou+ep_cardinality_error
    
    print('Saving model at epoch '+str(i)+', step '+str(_iter))
    print("5 Avg Losses: {0}, {1}, {2}, {3}, {4}".format(ep_loss_ce, ep_class_error, ep_loss_bbox, ep_loss_giou, ep_cardinality_error))
    validation_score=0
    torch.save(GWD_detr_Model.state_dict(), "./GWD_DETR_SD_Epoch_{0}_Score_{1:.4f}_EpLoss_{2:.4f}.pt".format(i, validation_score, _epoch_loss))
    torch.cuda.empty_cache()
    gc.collect() 


loss_ce:0.4735482, class_error:4.347824, loss_bbox:0.419187, loss_giou:1.0901135, cardinality_error:57.75, 


KeyboardInterrupt: 

# Inference

In [1]:
import os
import torch
import torchvision

from detr.config import config
from detr.utils.inferencefns import (get_detr, 
                                     WheatInferenceDataset, 
                                     collate_fn, 
                                     format_prediction_string, 
                                     make_submission_file, 
                                     post_process)
from global_utils.weighted_boxes_fusion.ensemble_boxes import nms, weighted_boxes_fusion

saved_model_dict= 'GWD_DETR_SD_Epoch_25_Score_0.0000_EpLoss_17.2091.pt'

In [2]:
conf = config()
#conf.DATA_PATH = os.path.join('/kaggle','input', 'global-wheat-detection')
conf.BATCH_SIZE = 2
conf.DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

WD_Inference = WheatInferenceDataset(conf, normalize=True)

In [3]:
num_classes=1
model = get_detr('resnet50', dilation=False, num_classes=num_classes)
#model.load_state_dict(torch.load('/kaggle/input/detrmodels/'+saved_model_dict, map_location=conf.DEVICE), strict=False)   #, strict=False
model.load_state_dict(torch.load('saved_weights/detr/GWD_DETR_SD_Epoch_42_EpLoss_20.1182_withaug_nomergeaug.pt'))
_ = model.eval()
_ = model.to(conf.DEVICE)

In [8]:
outputs = {idx:[] for idx in WD_Inference.IMGS}
scores_ = []
for i in range(len(WD_Inference)):
    image_id, image = WD_Inference[i]
    results = model([torch.tensor(image).to(conf.DEVICE)])
    
    #(Xcen, Ycen, Height, Width) - scale to 0-1
    boxes = results['pred_boxes'].cpu().detach().numpy()[0]
    raw_scores = results['pred_logits'].cpu().detach().numpy()[0]
    
    scores, boxes = post_process(raw_scores, boxes)
    
    # changing the boxes to (x1, y1, x2, y2) - scale to 0-1
    boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3] = boxes[:,0]-boxes[:,3]/2, boxes[:,1]-boxes[:,2]/2, boxes[:,0]+boxes[:,3]/2, boxes[:,1]+boxes[:,2]/2
    
    outputs[image_id].append({'boxes':boxes*1024, 'scores':scores, 'labels':np.ones(len(boxes))})

In [9]:
iou_thresh = 0.5
skip_box_thresh=0.1

fused_outputs = {}
for image_id in WD_Inference.IMGS:
    boxes, scores, labels = outputs[image_id][0]['boxes']/1024, outputs[image_id][0]['scores'], outputs[image_id][0]['labels']
    boxes, scores, labels = weighted_boxes_fusion([boxes], [scores], [labels], weights=[1], iou_thr=iou_thresh, skip_box_thr=skip_box_thresh)
    
    # Now I will multiply the boxes to original scale
    fused_outputs[image_id] = {'boxes': boxes*1024, 'scores': scores, 'labels': labels}

D:\Projects\Kaggle\Global-Wheat-Detction-Kaggle-2020\global_utils\weighted_boxes_fusion\ensemble_boxes\ensemble_boxes_wbf.py:64: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
D:\Projects\Kaggle\Global-Wheat-Detction-Kaggle-2020\global_utils\weighted_boxes_fusion\ensemble_boxes\ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
D:\Projects\Kaggle\Global-Wheat-Detction-Kaggle-2020\global_utils\weighted_boxes_fusion\ensemble_boxes\ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
D:\Projects\Kaggle\Global-Wheat-Detction-Kaggle-2020\global_utils\weighted_boxes_fusion\ensemble_boxes\ensemble_boxes_wbf.py:76: UserWarning: Y1 < 0 in box. 

In [12]:
strict_cut_off = 0.5
pred_strings = []
for img_id in WD_Inference.IMGS:
    try:
        stop_index = np.argwhere(fused_outputs[img_id]['scores']<strict_cut_off)[0][0]
    except:
        stop_index = len(fused_outputs[img_id]['scores'])
    
    pred_string = format_prediction_string(fused_outputs[img_id]['boxes'][:stop_index], scores[:stop_index])
    pred_strings.append(pred_string)
make_submission_file(WD_Inference.IMGS, pred_strings)

file_saved


In [1]:

from global_utils.model_precision import calculate_model_precision
import pandas as pd
import numpy as np
import ast


In [2]:
train_df = pd.read_csv(f'data/global-wheat-detection/train.csv')
box_data = np.stack(train_df['bbox'].apply(lambda x: ast.literal_eval(x)))
train_df[['x', 'y', 'w', 'h']] = pd.DataFrame(box_data).astype(np.float32)

In [3]:
sample_id = '1ef16dab1'

gt_boxes = train_df[train_df['image_id'] == sample_id][['x', 'y', 'w', 'h']].values
gt_boxes = gt_boxes.astype(np.int)

# Ground-truth boxes of our sample
gt_boxes

array([[954, 391,  70,  90],
       [660, 220,  95, 102],
       [ 64, 209,  76,  57],
       [896,  99, 102,  69],
       [747, 460,  72,  77],
       [885, 163, 103,  69],
       [514, 399,  90,  97],
       [702, 794,  97,  99],
       [721, 624,  98, 108],
       [826, 512,  82,  94],
       [883, 944,  79,  74],
       [247, 594, 123,  92],
       [673, 514,  95, 113],
       [829, 847, 102, 110],
       [ 94, 737,  92, 107],
       [588, 568,  75, 107],
       [158, 890, 103,  64],
       [744, 906,  75,  79],
       [826,  33,  72,  74],
       [601,  69,  67,  87]])

In [4]:
preds = np.array([[956, 409, 68, 85],
                  [883, 945, 85, 77],
                  [745, 468, 81, 87],
                  [658, 239, 103, 105],
                  [518, 419, 91, 100],
                  [711, 805, 92, 106],
                  [62, 213, 72, 64],
                  [884, 175, 109, 68],
                  [721, 626, 96, 104],
                  [878, 619, 121, 81],
                  [887, 107, 111, 71],
                  [827, 525, 88, 83],
                  [816, 868, 102, 86],
                  [166, 882, 78, 75],
                  [603, 563, 78, 97],
                  [744, 916, 68, 52],
                  [582, 86, 86, 72],
                  [79, 715, 91, 101],
                  [246, 586, 95, 80],
                  [181, 512, 93, 89],
                  [655, 527, 99, 90],
                  [568, 363, 61, 76],
                  [9, 717, 152, 110],
                  [576, 698, 75, 78],
                  [805, 974, 75, 50],
                  [10, 15, 78, 64],
                  [826, 40, 69, 74],
                  [32, 983, 106, 40]]
                )

scores = np.array([0.9932319, 0.99206185, 0.99145633, 0.9898089, 0.98906296, 0.9817738,
                   0.9799762, 0.97967803, 0.9771589, 0.97688967, 0.9562935, 0.9423076,
                   0.93556845, 0.9236257, 0.9102379, 0.88644403, 0.8808225, 0.85238415,
                   0.8472188, 0.8417798, 0.79908705, 0.7963756, 0.7437897, 0.6044758,
                   0.59249884, 0.5557045, 0.53130984, 0.5020239])

# Sort highest confidence -> lowest confidence
preds_sorted_idx = np.argsort(scores)[::-1]
preds_sorted = preds[preds_sorted_idx]

In [5]:
import numba
iou_thresholds = numba.typed.List()

for x in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75]:
    iou_thresholds.append(x)

In [9]:
calculate_model_precision([gt_boxes]*1000, [gt_boxes]*100, form='coco')

(0.9999999999999999,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [30]:
2500*12/1000

30.0